In [1]:
from keras.models import load_model
import numpy as np
model = load_model('candidateBC_1.h5')

In [2]:
W1 = model.layers[0].get_weights()[0]
b1 = model.layers[0].get_weights()[1]
#print(weights,biases)
W2 = model.layers[1].get_weights()[0]
b2 = model.layers[1].get_weights()[1]
W3 = model.layers[2].get_weights()[0]
b3 = model.layers[2].get_weights()[1]
W3_temp=np.transpose(W3)

In [3]:
n=10

In [4]:
import gurobipy as gp
import scipy

Define the bounds on state variables. 
This will be the Admissible region. Safety constraints will be explicitly enforced later

In [5]:
#Admissible Region

#lb=[-1,-1,-1,-1,0.4644,-1,-1,-1,-1,-1]
#ub=[1,1,1,1,0.4944,1,1,1,1,1]

lb=[0.646,-1e-3,0.646,-1e-3,0.35,-1e-3,0.646,-1e-3,0.3,-1e-3]
ub=[0.714,1e-3,0.714,1e-3,0.65,1e-3,0.714,1e-3,0.6,1e-3]

In [6]:
m=gp.Model('safe')

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-10


In [7]:
#Create decision variables corresponding to the states

x=m.addMVar(n,name='state',vtype=gp.GRB.CONTINUOUS,lb=lb,ub=ub)

In [8]:
# Enforcing  safe/unsafe region constraints
# initial states: 0.4644 <= vod <= 0.4944
# Unsafe region: 0.4944<= vod <= 0.65 or 0.35<= vod <= 0.4644
# for unsafe region 2 separate optimization problems

safe_region_constr_ub=m.addConstr(x[4]>=0.4644,"saferegion1")
safe_region_constr_lb=m.addConstr(x[4]<=0.4944,"saferegion2")

#Unsafe Region #1 
#unsafe_region_constr_ub=m.addConstr(x[4]>=0.4944,"unsaferegion1")
#unsafe_region_constr_lb=m.addConstr(x[4]<=0.65,"unsaferegion2")

#Unsafe Region#2
#unsafe_region_constr_ub=m.addConstr(x[4]>=0.35,"unsaferegion1")
#unsafe_region_constr_lb=m.addConstr(x[4]<=0.4644,"unsaferegion2")

In [9]:
## Big-M bounds 
lk=-1e4
uk=1e4

In [10]:
## Relu constraints Unrolled for each hidden layer

#Hidden layer temp variable
Z1=m.addMVar(n,vtype=gp.GRB.CONTINUOUS,name='z1')

#Z1=W1^T*x + b1
z_value_constr=m.addConstr(Z1==np.transpose(W1) @ x +b1,'Z1Value')

#Binary indicator variable
t1=m.addMVar(n,vtype=gp.GRB.BINARY, name="t1")

#Output of hidden layer 1 i.e. x1=ReLU(Z1)
x1=m.addMVar(n,vtype=gp.GRB.CONTINUOUS,name='x1')

### RElU constraints same as defined in SynthBC paper
relu_constr_1a=m.addConstr(x1-Z1+lk<=lk*t1,'relu_cons_1a')
relu_constr_1b=m.addConstr(Z1-x1 <= 0,'relu_cons_1b')
relu_constr_1c=m.addConstr(x1-uk*t1 <= 0,'relu_cons_1c')
relu_constr_1d=m.addConstr(x1 >= 0,'relu_cons_1d')

#2nd Hidden layer

#Hidden layer temp variables
Z2=m.addMVar(n,vtype=gp.GRB.CONTINUOUS,name='z2')

#Binary indicator variables
t2=m.addMVar(n,vtype=gp.GRB.BINARY, name="t2")

#Z2=W2^T*x1+b2
z_value_constr2=m.addConstr(Z2==np.transpose(W2) @ x1 +b2,'Z2Value')

#Output of hidden layer 2 i.e. x2=ReLU(Z2)
x2=m.addMVar(n,vtype=gp.GRB.CONTINUOUS,name='x2')

#ReLU constraints
relu_constr_2a=m.addConstr(x2-Z2+lk<=lk*t2,'relu_cons_2a')
relu_constr_2b=m.addConstr(Z2-x2 <= 0,'relu_cons_2b')
relu_constr_2c=m.addConstr(x2-uk*t2 <= 0,'relu_cons_2c')
relu_constr_2d=m.addConstr(x2 >= 0,'relu_cons_2d')


##Output layer: X3=W3^T*x2+b3
x3=m.addMVar(1,vtype=gp.GRB.CONTINUOUS,name='x3')
x3_constraint=m.addConstr(x3==W3_temp@x2+b3,'x3_val')



In [1]:
'''
def f(x):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10=x
    p_star=1
    q_star=1e-6
    w=60
    rn=1e3
    rc=0.0384
    cf=2500
    rf=2e-3
    lf=100e-6
    irefd=2.08
    irefq=1e-6
    kp=0.5
    vbd=0.48
    vbq=1e-6
    f1=[]
    f1.append(-p_star*x1+w*x2+vbd)
    f1.append(-p_star*x2-w*x1+vbq)
    f1.append(-rc*x3+w*x4+x5-vbd)
    f1.append((-rc*x4-w*x3+x6-vbq))
    f1.append((w*x6+(x7-x3)/cf))
    f1.append(-w*x5+(x8-x4)/cf)
    f1.append(-(rf/lf)*x7+w*x8+(x9-x5)/lf)
    f1.append(-(rf/lf)*x8-w*x7+(x10-x6)/lf);
    f1.append(-w*x8+kp*(irefd-x7))
    f1.append(-w*x7+kp*(irefq-x8))
    return f1
'''

'\ndef f(x):\n    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10=x\n    p_star=1\n    q_star=1e-6\n    w=60\n    rn=1e3\n    rc=0.0384\n    cf=2500\n    rf=2e-3\n    lf=100e-6\n    irefd=2.08\n    irefq=1e-6\n    kp=0.5\n    vbd=0.48\n    vbq=1e-6\n    f1=[]\n    f1.append(-p_star*x1+w*x2+vbd)\n    f1.append(-p_star*x2-w*x1+vbq)\n    f1.append(-rc*x3+w*x4+x5-vbd)\n    f1.append((-rc*x4-w*x3+x6-vbq))\n    f1.append((w*x6+(x7-x3)/cf))\n    f1.append(-w*x5+(x8-x4)/cf)\n    f1.append(-(rf/lf)*x7+w*x8+(x9-x5)/lf)\n    f1.append(-(rf/lf)*x8-w*x7+(x10-x6)/lf);\n    f1.append(-w*x8+kp*(irefd-x7))\n    f1.append(-w*x7+kp*(irefq-x8))\n    return f1\n'

In [11]:
#Derivative Condition constraints
#v=m.addMVar(10,name='V')
#x3_constraint=m.addConstr(W3_temp@x2+b3==0,'x3_val')
#v_constr=m.addConstr(v==np.multiply(W3,np.multiply(t2@W2,t1@W1)),'v_val')


In [12]:
## Set objective for Gurobi: 
#For initial states x3 should be -ve. Obj: p* =max(x3). if p* < 0 then BaC satisifies initial states condition
#For unsafe states x3 should be +ve. Obj: p*=min(x3). if p* > 0 then BaC satisfies unsafe region condition
m.setObjective(x3, gp.GRB.MAXIMIZE)
#m.setObjective(x3,gp.GRB.MINIMIZE)


#m.setObjective(v@f(x),gp.GRB.MAXIMIZE)

In [13]:
# Save the model
m.write("grb.lp")

In [14]:
# Call optimizer
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 103 rows, 71 columns and 393 nonzeros
Model fingerprint: 0xaec2fd05
Variable types: 51 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-03, 1e+00]
  RHS range        [2e-02, 1e+04]
Presolve removed 20 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


In [ ]:

m.computeIIS()
m.write('grp.ilp')

Following block of code, computes the IIS(Irreducible Inconsistent Subsystem)
An IIS is a subset of the constraints and variable bounds with the following properties:

It is still infeasible, and
If a single constraint or bound is removed, the subsystem becomes feasible.

if optimization problem is infeasible
    
    
    1. Compute IIS
    2. Remove one of the IIS constraints except safe/unsafe state constraints
    3. Keep track of removed Constraint
    4. Save the model
    5. Optimize the model

In [ ]:
removed=[]
while m.Status!=2:
    m.computeIIS()
    for c in m.getConstrs():
         if c.IISConstr and c.ConstrName.find('saferegion1')==-1 and c.ConstrName.find('saferegion2')==-1:
            print('%s' % c.ConstrName)
            removed.append(c.ConstrName)
            m.remove(c)
            
            break
    m.write('updated_model.lp')
    m.optimize()

In [ ]:
#m.x[0:10] gives states corresponding to p*.

model.predict([m.x[0:10]])

In [ ]:
m.x[0:10]

In [ ]:
removed

In [ ]:
#This relaxes the constraints, adds the cost of relaxation as an objective.Generates a different optimization problem
m.feasRelaxS(1, False, False, True)